## 1 Acessar site e coletar os PDFs

In [1]:
#%%
#instalar dependencias
%pip install beautifulsoup4

In [2]:
### Imports, variaveis globais 

import requests
from bs4 import BeautifulSoup

import re #regex
from datetime import datetime as dt
import pandas as pd

URL_CEASA = 'https://www.ceasa.rj.gov.br'
URL_CEASA_cotacao = URL_CEASA+'Cota%C3%A7%C3%A3o'

PASTA_PDFs = "./pdfs/"
PASTA_DADOS = "./dados/"

ceasa_lista_pdf = PASTA_DADOS+"ceasa_lista_pdf.csv"

##### auxs

In [23]:
# barra de progresso
# font:https://stackoverflow.com/questions/6169217/replace-console-output-in-python

def print_r(str:str):
    print(str, end='\r')


def print_percent_done(index, total, progress_state=None, bar_len=50, title='Please wait'):
    '''
    index is expected to be 0 based index. 
    0 <= index < total
    '''
    percent_done = (index+1)/total*100
    percent_done = round(percent_done, 1)

    done = round(percent_done/(100/bar_len))
    togo = bar_len-done

    done_str = '█'*int(done)
    togo_str = '░'*int(togo)

    progress_state = f"\n\t Stage: {progress_state}" if progress_state else ''

    print(f'\t⏳{title}: [{done_str}{togo_str}] {percent_done}% done {progress_state}', end='\r')

    if round(percent_done) == 100:
        print('\t✅')


### Coleta linear

In [90]:
urls = df_extração["URL"].tolist()

In [ ]:
def foi_extraida(url):
	return True if url in((df_extração['URL'].eq(url))) else False

def pegar_links(url):
	"""
		Função recursiva que entra no site em busca links das tags 'a' na div com id="main" e entra no novo site em busta de links. ele para quando encrontra uma url com '.pdf'.

		Args:
			url (str):endereço da pagina html a ser percorrida
				(opcional)

		Return:
			None
	"""
	
	reqs = requests.get(url)
	soup = BeautifulSoup(reqs.text, 'html.parser')

	for link in soup.select('#main a'):
		
		a = str(link.get('href'))
		print_r(f'Qtds pdfs encontrados {len(urls)}, link atual: {a}')
		if ".pdf" in a:
			a = 'https://www.ceasa.rj.gov.br'+ a
			if not foi_extraida(a):
				continue
			urls.append(a)

		else:
				pegar_links(a)

def extrair_dados_url(url):
	"""
			Pega a url do PDF e extrai data, nome do documento e link do pdf

		Args:
			link: str
				url do documento pdf

		Return:
			url: str
			nome_arquivo: str
			data: str
	"""
	# pega nome do arquivo na URL
	nome_arquivo = url.split("/")[-1]
	nome_arquivo = requests.utils.unquote(nome_arquivo)

	# padrao de dd mm yyyy para data
	matches = re.findall(r'(\d{2})\s(\d{2})\s(\d{4})', nome_arquivo)
	data = dt.strptime("/".join(matches[0]), "%d/%m/%Y")

	return url,"CEASA-RJ_" + nome_arquivo.replace(" ", "_") ,data.strftime("%d-%m-%Y")

def pegar_pdf(url,nome_arquivo):
	# conferir se ja está na base local
	response = requests.get(url)

	if response.status_code == 200:
		with open(PASTA_PDFs+nome_arquivo, "wb") as f:
			f.write(response.content)
		
		return True

def add_csv(url, nome_arquivo,data):
	data_to_file = {
		'URL': [url],
		'nome_arquivo': [nome_arquivo],
		'data': [data]
	} 
	df = pd.DataFrame(data_to_file)
	df.to_csv(ceasa_lista_pdf, mode='a', index=False, header=False, sep=";")

def atualizar_base_pdf():
	pegar_links(URL_CEASA_cotacao)
	
	for x in urls[:]:
		print_percent_done(
		len(df_extração) - len(urls), 
		len(df_extração), 
		str(f'pdf extraidos {len(urls)} de {len(df_extração)} \n\t link atual {x}'))

		# if foi_extraida:
		# 	continue
		
		url, nome_arquivo, data = extrair_dados_url(x)
		pegar_pdf(url,nome_arquivo)
		add_csv(url,nome_arquivo,data)


# init

try:
    urls 
except:
	urls = [] 


try:
	df_extração = pd.read_csv(ceasa_lista_pdf,sep=";")
except:
	df_extração = pd.read_csv(ceasa_lista_pdf,sep=";", names = ["URL", "nome_arquivo","data"])
	df_extração.to_csv(ceasa_lista_pdf, sep=';', index=False)

atualizar_base_pdf()


#### request async

<!-- #region skip -->

In [ ]:
#%%
!pip install aiohttp

In [ ]:
#%%
import re #regex
from datetime import datetime as dt
import pandas as pd

import aiohttp
import asyncio



PASTA_PDFs = "./pdfs/"
PASTA_DADOS = "./dados/"


def extrair_dados_url(url):
  """
      Pega a url do PDF e extrai data, nome do documento e link do pdf

    Args:
      link: str
        url do documento pdf

    Return:
      url: str
      nome_arquivo: str
      data: str
  """
  # pega nome do arquivo na URL
  nome_arquivo = url.split("/")[-1]
  nome_arquivo = requests.utils.unquote(nome_arquivo)

  # padrao de dd mm yyyy para data
  matches = re.findall(r'\b(\d{1,2})\s(\d{1,2})\s(\d{4})\b', nome_arquivo)
  data = dt.strptime("/".join(matches[0]), "%d/%m/%Y")

  return url,"CEASA-RJ_" + nome_arquivo.replace(" ", "_") ,data.strftime("%d-%m-%Y")

async def pegar_pdf(resp, url,nome_arquivo):
  response = await resp

  if response.status_code == 200:
    # lista_url[i] = [lista_url[i],nome_arquivo]
    with open(PASTA_PDFs+nome_arquivo, "wb") as f:
      f.write(response.content)
      return True

async def add_csv(url, nome_arquivo,data):
  """
  Gera um arquivo relativo a coleta de pdfs

  Args:
    Tupla: tupla
      recebe uma tupla com url, nome do arquivo e data
  """

  # nome dos arquivos
  with open(PASTA_DADOS+"ceasa_lista_pdf.csv", "a") as f:
    if url is None:
      f.write("URL;Nome_Arquivo;Data\n")
    else:
      f.write(url)
      f.write(";")
      f.write(nome_arquivo )
      f.write(";")
      f.write(str(data))
      f.write('\n')

# async orquestra

async def requsicao_API(session, url):
    async with session.get(url,timeout=60*100) as resp:
        
        try:
            metadados = extrair_dados_url(x)
            url,nome_arquivo,data = metadados
            pegar_pdf(resp,url,nome_arquivo)
            add_csv(url,nome_arquivo,data)
                        
        except Exception as e:
            await print("requsicao_API", e,url)   


async def main():
  
  urls = urls[:2]

  session_timeout =   aiohttp.ClientTimeout(total=None,sock_connect=20 ,sock_read=20)
  connector = aiohttp.TCPConnector(limit_per_host=5)
  async with aiohttp.ClientSession(headers=None,timeout=session_timeout, connector=connector) as session:
      tasks = []      
      
      for url in urls:
          tasks.append(asyncio.ensure_future(requsicao_API(session, url)))

      try:
          await asyncio.gather(*tasks)

      except Exception as e:
          await print("main",e,'url')



# for x in urls[:2]:
#   metadados = extrair_dados_url(x)
#   print(metadados)
#   url,nome_arquivo,data = metadados
#   pegar_pdf(url,nome_arquivo)
#   add_csv(url,nome_arquivo,data)



# asyncio.run(main())
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

<!-- #endregion-->

## Tratando os PDF

In [ ]:
# instalando dependencias 
%pip install PyPDF2
%pip install pdfplumber

### compactando PDF (muito custoso) 

#### compresão serial

In [ ]:
# compactando PDF 

import PyPDF2

def compress_pdf(input_path, output_path):
    with open(input_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader (file)
        pdf_writer = PyPDF2.PdfWriter ()

        for page_num in range(len(pdf_reader.pages)):
            # page = pdf_reader.getPage(page_num)
            page = pdf_reader.pages[page_num]
            page.compress_content_streams()  # Comprime o conteúdo da página
            pdf_writer.add_page(page)

        with open(output_path, 'wb') as output_file:
            pdf_writer.write(output_file)

count = 0
for x in df_extração["nome_arquivo"]:
    print_percent_done(count, len(df_),str(x))
    
    input_pdf = x
    output_pdf = x
    print_r( f'arqiuvo atual: {input_pdf}')
    compress_pdf(PASTA_PDFs +  input_pdf, PASTA_PDFs +  output_pdf)



#### Compressão paralelizada

In [ ]:
import PyPDF2
import concurrent.futures

def compress_pdf(input_path, output_path):
    with open(input_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        pdf_writer = PyPDF2.PdfWriter()

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page.compress_content_streams()
            pdf_writer.add_page(page)

        with open(output_path, 'wb') as output_file:
            pdf_writer.write(output_file)

def process_pdf(file_name):
    print(f'arquivo atual: {file_name}')
    compress_pdf(PASTA_PDFs + file_name, PASTA_PDFs + file_name)

# Sua lista de arquivos
files_to_process = df_extração["nome_arquivo"].tolist()

# Número máximo de threads (ajuste conforme necessário)
max_threads = 4

# Usando ThreadPoolExecutor para paralelizar
with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    futures = [executor.submit(process_pdf, file_name) for file_name in files_to_process]

    # Esperar que todas as threads concluam
    for future in concurrent.futures.as_completed(futures):
        count += 1
        print_percent_done(count, len(files_to_process), future.result())


### Lendo Tabelas 

In [31]:
import pdfplumber

arq = ceasa_lista_pdf

df_extração = pd.read_csv(arq)

pdf_v1 = PASTA_PDFs + "CEASA-RJ_Boletim_diário_de_preços__03_01_2022_0.pdf" 
pdf_v2 = PASTA_PDFs + "CEASA-RJ_Boletim_diário_de_preços__24_04_2023.pdf" 



def extrair_tabelas(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        # Itera sobre todas as páginas do PDF
        for page_number in range(len(pdf.pages)):
            page = pdf.pages[page_number]

            # Extrai todas as tabelas na página
            tables = page.extract_tables()

            # Itera sobre todas as tabelas na página
            for table_number, table in enumerate(tables):
                print(f"Tabela {table_number + 1} na página {page_number + 1}:")
                
                # Itera sobre todas as linhas da tabela
                for row_number, row in enumerate(table):
                    print(f"  Linha {row_number + 1}: {row}")

                print("\n" + "="*50 + "\n")  # Separador entre tabelas


def extrair_tabela_pdf(pdf_path, pagina=1):
    # Abre o PDF com pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        # Seleciona a página desejada
        page = pdf.pages[pagina]
        
        # Extrai a tabela como uma lista de dicionários
        table_data = page.extract_table()

        # Converte a lista de dicionários para um DataFrame do pandas
        df = pd.DataFrame(table_data)

    return df

df = extrair_tabela_pdf(pdf_v2)


In [32]:
df

,0,1,2,3,4,5,6
0,None,Dia Semana:,segunda-feira,None,None,DATA:,24/04/2023
1,PRODUTOS,TIPO,UNIDADE EMBALAGEM,VARIAÇÃO\nULTIMOS\n12 MESES,MIN,MODAL,MAX
2,SERIGUELA,,Cx 1 kg,"33,33%","30,00","40,00","40,00"
3,TAMARINDO,,Cx 2 kg,"33,33%","30,00","40,00","40,00"
4,TANGERINA COMUM / RIO,Extra,Cx 10 kg,"-20,00%","50,00","60,00","70,00"
5,TANGERINA MURCOTT,Extra,Cx 9 kg,"6,25%","75,00","85,00","90,00"
6,TANGERINA PONKAN,Extra,Cx 9 kg,"66,67%","70,00","75,00","85,00"
7,UVA BENITAKA,Extra,Cx 5 kg,"20,00%","50,00","60,00","70,00"
8,UVA ITALIA,Extra,Cx 5 kg,"40,70%","60,00","70,00","80,00"
9,UVA RED GLOBE NACIONAL,,Cx 5 kg,"-38,89%","50,00","55,00","65,00"


fazendo tratamento na tabela 1 do pdf modelo 2

In [ ]:
"""
Cell generated by Data Wrangler.
"""
# tratamento v2 pag 1 tb 2
def clean_data(df):
    df = df.iloc[1: , :]
    df.columns = df.iloc[0]
    # Change column type to string for column: 'PRODUTOS'
    df = df.astype({'PRODUTOS': 'string'})
    df = df.iloc[1: , :]
    df['CLASSE'] = df['PRODUTOS'].iloc[0]
    df = df.iloc[1: , :]
    # Replace all instances of "" with "0" in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace("^$", "0", regex=True)
    df['MODAL'] = df['MODAL'].str.replace("^$", "0", regex=True)
    df['MAX'] = df['MAX'].str.replace("^$", "0", regex=True)
    # Replace all instances of "" with "S/C" in column: 'UNIDADE EMBALAGEM'
    df.loc[df['UNIDADE EMBALAGEM'].str.lower() == "".lower(), 'UNIDADE EMBALAGEM'] = "S/C"
    # Replace all instances of "," with "." in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace(",", ".", case=False, regex=False)
    df['MODAL'] = df['MODAL'].str.replace(",", ".", case=False, regex=False)
    df['MAX'] = df['MAX'].str.replace(",", ".", case=False, regex=False)
    # Change column type to float16 for columns: 'MIN', 'MODAL', 'MAX'
    df = df.astype({'MIN': 'float16', 'MODAL': 'float16', 'MAX': 'float16'})
    # Change column type to string for columns: 'CLASSE', 'PRODUTOS' and 2 other columns
    df = df.astype({'CLASSE': 'string', 'PRODUTOS': 'string', 'TIPO': 'string', 'UNIDADE EMBALAGEM': 'string'})
    # Convert text to uppercase in column: 'TIPO'
    df['TIPO'] = df['TIPO'].str.upper()
    return df

df_clean = clean_data(df.copy())
df_clean.head()